In [14]:
# Imports
from pathlib import Path
import glob
import json
import os.path

from gat_vqa.config.gqa import GQASplit, GQAVersion


In [15]:
# Files Loading
def load_file(name):
    """Load a file."""
    print(name)
    # load standard json file
    if os.path.isfile(name):
        with open(name) as file:
            data = json.load(file)
    # load file chunks if too big
    elif os.path.isdir(os.path.dirname(name)):
        dir_, ext = os.path.splitext(os.path.basename(name))
        data = {}
        chunks = glob.glob(
            os.path.join(
                os.path.dirname(name), "{dir}/{dir}_*{ext}".format(dir=dir_, ext=ext)
            )
        )
        print(chunks)
        for chunk in chunks:
            with open(chunk) as file:
                data.update(json.load(file))
    else:
        raise Exception("Can't find {}".format(name))
    return data

In [16]:
tiers = [split.value for split in GQASplit]
for tier in tiers:
    ### TODO remove when we have more memory
    if tier == "train":
        continue
    ###

    question_path = f"data/gqa/questions/{tier}_all_questions.json"
    questions = load_file(question_path)
    img_ids = list(set([q_data['imageId'] for q_data in questions]))
    with open(f"./ids/{tier}.json", "w") as f:
        json.dump(img_ids, f)

data/gqa/questions/val_all_questions.json
